# Installing Necessary Libraries

In [1]:
!pip install geopandas

# Importing Libraries

In [2]:
import numpy as np
import pandas as pd
import geopandas as gpd
import json
import csv

import matplotlib.pyplot as plt
from shapely.geometry import box, Polygon, LineString, Point

import os

# Example Code from Dr. Griffin

In [3]:
# pandas data frame from cites in a csv
df = pd.read_csv("ufo_data.csv")

df.head()

,city,state,shape,duration,date_time,lon,lat
0,Visalia,CA,light,2 minutes,2021-12-15T21:45:00,-119.347937,36.356650
1,Cincinnati,OH,triangle,14 seconds,2021-12-16T09:45:00,-84.481363,39.174503
2,Knoxville,TN,triangle,20-30 seconds,2021-12-10T19:30:00,-83.980115,35.961561
3,Fullerton,CA,unknown,2 minutes,2020-07-07T23:00:00,-117.924978,33.877422
4,Las Vegas,NV,oval,10 minutes,2020-04-23T03:00:00,-115.186592,36.141246


In [4]:
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat))
gdf.set_crs(epsg=4326, inplace=True)
gdf.head()

,city,state,shape,duration,date_time,lon,lat,geometry
0,Visalia,CA,light,2 minutes,2021-12-15T21:45:00,-119.347937,36.356650,POINT (-119.34794 36.35665)
1,Cincinnati,OH,triangle,14 seconds,2021-12-16T09:45:00,-84.481363,39.174503,POINT (-84.48136 39.17450)
2,Knoxville,TN,triangle,20-30 seconds,2021-12-10T19:30:00,-83.980115,35.961561,POINT (-83.98012 35.96156)
3,Fullerton,CA,unknown,2 minutes,2020-07-07T23:00:00,-117.924978,33.877422,POINT (-117.92498 33.87742)
4,Las Vegas,NV,oval,10 minutes,2020-04-23T03:00:00,-115.186592,36.141246,POINT (-115.18659 36.14125)


# Opening/Converting Data Files

In [5]:
# open city file
with open("cities.geojson", 'r') as cities:
  cityData = json.load(cities)

In [6]:
cityData

{'features': [{'geometry': {'coordinates': [-122.676482, 45.523062],
    'type': 'Point'},
   'properties': {'city': 'Portland',
    'growth': 15,
    'marker-color': '#879CB9',
    'marker-size': 'medium',
    'population': 609456,
    'rank': 29,
    'state': 'Oregon'},
   'type': 'Feature'},
  {'geometry': {'coordinates': [-122.332071, 47.606209], 'type': 'Point'},
   'properties': {'city': 'Seattle',
    'growth': 15.6,
    'marker-color': '#14F334',
    'marker-size': 'medium',
    'population': 652405,
    'rank': 21,
    'state': 'Washington'},
   'type': 'Feature'},
  {'geometry': {'coordinates': [-118.243685, 34.052234], 'type': 'Point'},
   'properties': {'city': 'Los Angeles',
    'growth': 4.8,
    'marker-color': '#5F2B67',
    'marker-size': 'medium',
    'population': 3884307,
    'rank': 2,
    'state': 'California'},
   'type': 'Feature'},
  {'geometry': {'coordinates': [-116.214607, 43.61871], 'type': 'Point'},
   'properties': {'city': 'Boise City',
    'growth': 9.5

In [7]:
# list for storing UFO sightings
ufoSightingList = []

# open and convert csv to dictionary
with open('ufo_data.csv') as ufoFile:
  ufoDict = csv.DictReader(ufoFile)

  # extract only coordinates from dictionary
  for sighting in ufoDict:
    latLon = float(sighting['lon']), float(sighting['lat'])
    ufoSightingList.append(Point(latLon));

In [8]:
len(ufoSightingList)

100925

In [26]:
# lists for storing city names and coordinates
cities = []
cityCoords = []

for feature in cityData["features"]:
  if feature["geometry"]["type"] == "Point":
    cityCoords.append(feature["geometry"]["coordinates"])
    cities.append(feature["properties"]["city"])

In [27]:
cities 

['Portland',
 'Seattle',
 'Los Angeles',
 'Boise City',
 'Las Vegas',
 'Phoenix',
 'Salt Lake City',
 'Billings',
 'Albuquerque',
 'Denver',
 'Cheyenne',
 'Oklahoma City',
 'Wichita',
 'Fargo',
 'Sioux Falls',
 'Omaha',
 'Houston',
 'Kansas City',
 'Des Moines',
 'Minneapolis',
 'Little Rock',
 'Jackson',
 'New Orleans',
 'Memphis',
 'Milwaukee',
 'Chicago',
 'Birmingham',
 'Indianapolis',
 'Louisville/Jefferson County',
 'Atlanta',
 'Detroit',
 'Columbus',
 'Jacksonville',
 'Charleston',
 'Columbia',
 'Charlotte',
 'Washington',
 'Baltimore',
 'Virginia Beach',
 'Wilmington',
 'Philadelphia',
 'Newark',
 'New York',
 'Burlington',
 'Bridgeport',
 'Manchester',
 'Providence',
 'Boston',
 'Portland']

In [22]:
len(cities)

49

In [28]:
len(cityCoords)

49

In [ ]:
cityCoords

In [30]:
cityPoints = []

# convert cityCoords elements (lists) into Points
for coord in cityCoords:
  cityPoints.append(Point(coord))

# convert shapely Points to geoseries
geoCityPoints = gpd.GeoSeries(cityPoints)

In [31]:
len(cityPoints)

49

In [32]:
len(geoCityPoints)

49

# Calculating Distances between Cities

In [46]:
# list of distances to be dumped into .json
cityDistOutputList = []

In [55]:
# Each city gets iterated over
for cityPoint in range(len(geoCityPoints)):
# for cityPoint in range(2):
  distCity2CityList = []

  distances = geoCityPoints.distance(geoCityPoints[cityPoint])
  for i, dist in enumerate(distances):
    # print(distances.values[i])
    distCity2CityList.append([cities[i], distances.values[i]])

  # store distances to other cities for each city
  output = {
      'City': cities[cityPoint],
      'distance': distCity2CityList,
      'coordinates': [geoCityPoints[cityPoint].x, geoCityPoints[cityPoint].y],
  }

  cityDistOutputList.append(output)

In [ ]:
distances

In [57]:
len(distances)

49

In [ ]:
cityDistOutputList

# Write Distances to File

In [59]:
with open('CityDistanceFile.json', 'w') as outFile:
  outFile.write(json.dumps(cityDistOutputList))

# Average Distance to 100 closest UFO Sightings





In [71]:
import statistics # helpful for calculating average

averageUFODistList = []

In [72]:
# Convert UFO sightings to geoseries
geoUFO = gpd.GeoSeries(ufoSightingList)

for cityPoint in range(len(geoCityPoints)):
  distances = geoUFO.distance(geoCityPoints[cityPoint])
  distances = distances.values
  distances = np.sort(distances)
  closeDistances = distances[:100]
  avg = round(statistics.mean(closeDistances),2)

  output = {
      'City': cities[cityPoint],
      'average UFO distance': avg,
      'coordinates': [geoCityPoints[cityPoint].x, geoCityPoints[cityPoint].y]
  }

  averageUFODistList.append(output)



In [68]:
len(distances)

100925

In [69]:
avg

0.09

# Write Average UFO Distances to File

In [74]:
with open('UFOAvgDistanceFile.json', 'w') as outFile:
  outFile.write(json.dumps(averageUFODistList))